# Musical Pattern Recognition

**Goal:** Identify repeating patterns (bars/phrases/sections)

In [ ]:
# Loads the autoreload extension
%load_ext autoreload
# Automatically reloads all modules before executing any code
%autoreload 2

In [ ]:
# External Imports
import matplotlib.pyplot as plt
import torch as torch
import numpy as np
import pypianoroll as pr

# Internal Imports
import sys, os
sys.path.append(os.path.abspath('src'))

from src.util.types import Song, PianoState, NoteSample, PianoStateSamples
from src.util.globals import resolution, beats_per_bar, num_pitches
from src.util.process_audio import quantize_pianoroll
import src.util.plot as plot


from src.dataset.load import (
    load_multi_track,
    get_track_by_instrument,
)

In [ ]:
# Testing Sample
dir_id = 'TRAAAGR128F425B14B'
song_id = 'b97c529ab9ef783a849b896816001748'

desired_instrument = 'Bass'

# EXAMPLE: Load a NPZ file into a Multitrack object.
multi_track = load_multi_track(f'A/A/A/TRAAAGR128F425B14B/b97c529ab9ef783a849b896816001748.npz')
bass_track = get_track_by_instrument(multi_track, desired_instrument)

# Binarize Note Velocity!
binary_track = bass_track.binarize()

track_pr = binary_track.pianoroll.astype(int)

print('unique values =', np.unique(track_pr))

# Quantize!
track_pr = quantize_pianoroll(track_pr, resolution//2)


plot.plot_pianoroll(track_pr, tick_resolution=resolution*8)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score

def compute_similarities_sklearn(windows):
    """
    Using scikit-learn's optimized implementation
    """
    # Reshape windows to 2D (num_windows, num_features)
    if windows.ndim == 3:  # (num_windows, time, pitch)
        windows_flat = windows.reshape(windows.shape[0], -1)
    else:
        windows_flat = windows

    # Compute cosine similarity matrix
    similarity_matrix = cosine_similarity(windows_flat)

    return similarity_matrix

def find_repeating_sections(piano_roll: np.ndarray, window_size: int):
    """
    Find repeating sections in piano roll

    Args:
        piano_roll: 2D array (time, pitch)
        X_bars: number of bars per window
        beats_per_bar: typically 4
        resolution: time steps per beat (4 = 16th notes)
    """

    # Calculate window size
    step_size = beats_per_bar  # Step by 1 bar

    # Sliding window view
    windows: np.ndarray = np.lib.stride_tricks.sliding_window_view(
        piano_roll,
        window_shape=(window_size, piano_roll.shape[1]),
        axis=(0, 1)
    )[::step_size, 0]  # Take every step_size-th window

    # Flatten windows for similarity computation
    windows_flat = windows.reshape(windows.shape[0], -1)

    # Compute similarity matrix
    similarity_matrix = cosine_similarity(windows_flat)

    return len(windows), similarity_matrix

def plot_sim_matrix(SM):
	plt.figure(figsize=(10, 8))
	plt.imshow(SM, cmap='hot', interpolation='nearest')
	plt.colorbar(label='Cosine Similarity')
	plt.title('Self-Similarity Matrix')
	plt.xlabel('Window Index')
	plt.ylabel('Window Index')
	plt.show()

def find_similair_pairs(sm: np.ndarray, threshold=0.99) -> list[tuple[int,int,float]]:
    ''' Find similar pairs (excluding diagonal) '''
    similar_pairs = []
    for i in range(len(sm)):
        for j in range(i+1, len(sm)):
            if sm[i,j] > threshold:
                similar_pairs.append((i, j, sm[i,j]))
    return similar_pairs

# Create groups:
def create_groups(sim_pairs, n_windows):
    groups = []
    window_groups = [-1] * n_windows  # assign each bar to a group

    for a, b, similarity in sim_pairs:
        if window_groups[a] == -1 and window_groups[b] == -1:
            # Both items are unassigned - create new group
            group_id = len(groups)
            window_groups[a] = group_id
            window_groups[b] = group_id
            groups.append([a, b])

        elif window_groups[a] == -1 and window_groups[b] != -1:
            # Add a to b's group
            groups[window_groups[b]].append(a)
            window_groups[a] = window_groups[b]

        elif window_groups[a] != -1 and window_groups[b] == -1:
            # Add b to a's group
            groups[window_groups[a]].append(b)
            window_groups[b] = window_groups[a]

        elif window_groups[a] != window_groups[b]:
            # Both belong to different groups - merge them
            group_a = window_groups[a]
            group_b = window_groups[b]

            # Move all items from group_b to group_a
            groups[group_a].extend(groups[group_b])

            # Update group assignments for all items that were in group_b
            for item in groups[group_b]:
                window_groups[item] = group_a

            # Clear group_b
            groups[group_b] = []

        # If window_groups[a] == window_groups[b] and both != -1,
        # they're already in the same group - do nothing

    # Filter out empty groups
    final_groups = [group for group in groups if group]

    return final_groups

In [ ]:
window_resolution = (16 * resolution) * 2

n_windows, sim_matrix = find_repeating_sections(
	track_pr,
	window_resolution
)

plot_sim_matrix(sim_matrix)

sim_pairs = find_similair_pairs(sim_matrix, threshold=0.95)

groups = create_groups(sim_pairs, n_windows)
print(f'Found {len(groups)} unique groups:')
print(
	*groups,
	sep='\n'
)

In [ ]:
import matplotlib.colors as mcolors

def plot_track(data: pr.Track, desired_instrument: str, window_resolution: int, groups: list):

	total_time = data.pianoroll.shape[0]
	num_bars = int(total_time / beats_per_bar)
	num_windows = int(total_time / window_resolution)

	print(f'Num bars = {num_bars}')
	print(f'Num windows = {num_windows}')

	# plot the track (with bars)
	fig, ax = plt.subplots(figsize=(12, 6))
	pr.plot_track(data, ax=ax)

	# Define more distinct colors for different groups
	n = len(groups)
	colors = [mcolors.to_hex(mcolors.hsv_to_rgb([i/n, 0.8, 0.9])) for i in range(n)]

	# Create a mapping from window index to group color
	window_to_color = {}
	for group_idx, group in enumerate(groups):
		for window_idx in group:
			window_to_color[window_idx] = colors[group_idx]

	# Color background regions for each WINDOW based on groups
	for window_idx in range(num_windows):
		# Calculate window boundaries (not bar boundaries)
		i0 = window_idx * window_resolution
		i1 = (window_idx + 1) * window_resolution

		ax.axvline(i0)
		ax.axvline(i1)

		color = window_to_color.get(window_idx, None)
		if color is not None:
			ax.axvspan(i0, i1, color=color, alpha=0.3, zorder=0)


	# Add labels
	title = f'{desired_instrument} Track - {num_bars} bars, {num_windows} windows of size {window_resolution} '
	if groups is not None:
		title += f' ({len(groups)} groups)'
	ax.set_title(title)
	ax.set_xlabel('Time (beats)')

	# Add legend if groups are provided
	legend_elements = []
	for group_idx, group in enumerate(groups):
		color = colors[group_idx]
		legend_elements.append(plt.Line2D([0], [0], color=color, lw=4,
										label=f'Group {group_idx} ({len(group)} windows)'))
	ax.legend(handles=legend_elements, loc='upper right')

	plt.tight_layout()
	plt.show()

In [ ]:
plot_track(binary_track, desired_instrument, window_resolution, groups)